# Import

In [1]:
import os
import sys

import json
import math
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import pytransform3d
from pytransform3d.urdf import UrdfTransformManager
from pytransform3d.plot_utils import make_3d_axis
from urdf_parser_py.urdf import URDF

# Helper Functions

In [2]:
%matplotlib widget

def load_json(filename: str):
    # Construct the absolute path by joining the current directory and relative path
    absolute_path = os.path.join(os.getcwd(), '..', filename)
    # Load the JSON data
    with open(absolute_path, 'r') as file:
        json_data = json.load(file)    
    return json_data

def xml_to_str(robot):
    # To XML string with filtering
    temp_str = robot.to_xml_string()
    words = temp_str.split()
    words[5] = '>'
    urdf_str = ' '.join(words)
    return urdf_str

# Intialization

In [3]:
# Load Configs
cam_mtxs = load_json('camera_mtx.json')
gaze_ctrs = load_json('calib_params.json')
urdf_path = os.path.join(os.getcwd(),"..","urdf","chest_grace.urdf")
robot = URDF.from_xml_file(urdf_path)

# URDF

# URDF Parameters Edit

In [4]:
# URDF Variable Assignment
for joint in robot.joints:
    if joint.name == 'torso':
        joint.origin.position[0] = -0.12234 + 0  # Offset
        joint.origin.position[1] = -0.0325 + 0
        joint.origin.position[2] = 0.05692 + 0
    elif joint.name == 'neck_pitch':
        joint.origin.rotation[0] = 0 + 0
    elif joint.name == 'head_pitch':
        joint.origin.position[2] = 0.13172 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[2] = 0 + 0
    elif joint.name == 'eyes_pitch':
        joint.origin.position[0] = 0.08492 + 0
        joint.origin.position[2] = 0.05186 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[2] = 0 + 0
    elif joint.name == 'lefteye_yaw':
        joint.origin.position[1] = 0.02895 + 0
    elif joint.name == 'righteye_yaw':
        joint.origin.position[1] = -0.02895 + 0
    elif joint.name == 'lefteye_cam':
        joint.origin.position[0] = 0.015 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[1] = 0 + 0.03457662014401592*(-1)  # For Y, Downwards is + because of Camera Opencv Coordinate System
        joint.origin.rotation[2] = 0 + 0.18913797017088096  # Orientation Offset: Right Hand Rule (Positive CCW)
    elif joint.name == 'righteye_cam':
        joint.origin.position[0] = 0.015 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[1] = 0 + 0.05553511593216458*(-1)  # For Y, Downwards is + because of Camera Opencv Coordinate System
        joint.origin.rotation[2] = 0 + -0.2544206094843146  # Orientation Offset: Right Hand Rule (Positive CCW)
        
# XML to String
urdf_str = xml_to_str(robot)

## Visualize URDF

In [5]:
tm = UrdfTransformManager()
joints_list = ["world","torso","realsense","neck","neck_py","head","eyes","lefteye","righteye","leftcamera","rightcamera"]

In [6]:
tm.load_urdf(urdf_str)

In [7]:
# Visualize
plt.figure()
ax = make_3d_axis(1, 111, unit="m")
ax = tm.plot_frames_in("robot", whitelist=joints_list,
                       s=0.1, show_name=False, ax=ax)
ax = tm.plot_connections_in("robot", ax=ax)
plt.show()

## Setting the Joints

In [8]:
# Motor Cmd Value

cmd_lnp = math.radians(10)
cmd_lnt = math.radians(10)
cmd_unt = math.radians(10)
cmd_et = math.radians(10)
cmd_lep = math.radians(5)
cmd_rep = math.radians(5)

In [9]:
# Transforms

neck_pitch = cmd_lnt*0.5 + 0
neck_yaw = cmd_lnp*0.5 + 0
head_pitch = cmd_unt*0.5 + 0
eyes_pitch = cmd_et*0.4 + 0
lefteye_yaw = cmd_lep*1.6 + 0
righteye_yaw = cmd_rep*1.7 + 0

In [10]:
# Setting the Joints

tm.set_joint('neck_pitch', neck_pitch)
tm.set_joint('neck_yaw', neck_yaw)
tm.set_joint('head_pitch', head_pitch)
tm.set_joint('eyes_pitch', eyes_pitch)
tm.set_joint('lefteye_yaw', lefteye_yaw)
tm.set_joint('righteye_yaw', righteye_yaw)

In [11]:
# Visualize
plt.figure()
ax = make_3d_axis(1, 111, unit="m")
ax = tm.plot_frames_in("robot", whitelist=joints_list,
                       s=0.1, show_name=False, ax=ax)
ax = tm.plot_connections_in("robot", ax=ax)
plt.show()

In [12]:
# Function Prototype

def motor_cmd_joint(cmd_lnp,cmd_lnt,cmd_unt,cmd_et,cmd_lep,cmd_rep):  # degrees
    cmd_lnp = math.radians(cmd_lnp)
    cmd_lnt = math.radians(cmd_lnt)
    cmd_unt = math.radians(cmd_unt)
    cmd_et = math.radians(cmd_et)
    cmd_lep = math.radians(cmd_lep)
    cmd_rep = math.radians(cmd_rep)

    neck_pitch = cmd_lnt*0.5 + 0
    neck_yaw = cmd_lnp*0.5 + 0
    head_pitch = cmd_unt*0.5 + 0
    eyes_pitch = cmd_et*0.4 + 0
    lefteye_yaw = cmd_lep*1.6 + 0
    righteye_yaw = cmd_rep*1.7 + 0

    res_dict = {
        "neck_pitch": neck_pitch,
        "neck_yaw": neck_yaw,
        "head_pitch": head_pitch,
        "eyes_pitch": eyes_pitch,
        "lefteye_yaw": lefteye_yaw,
        "righteye_yaw": righteye_yaw,
    }

    return res_dict


In [13]:
# Testing the Function

motor_cmd_joint(cmd_lnp=10,cmd_lnt=10,cmd_unt=10,cmd_et=10,cmd_lep=5,cmd_rep=5)

{'neck_pitch': 0.08726646259971647,
 'neck_yaw': 0.08726646259971647,
 'head_pitch': 0.08726646259971647,
 'eyes_pitch': 0.06981317007977318,
 'lefteye_yaw': 0.13962634015954636,
 'righteye_yaw': 0.148352986419518}

# Camera Angle Rotation Offset

## Undistort Points

In [14]:
# Gaze Centers

left_gaze_pt_x = gaze_ctrs['left_eye']['x_center']
left_gaze_pt_y = gaze_ctrs['left_eye']['y_center']
right_gaze_pt_x = gaze_ctrs['right_eye']['x_center']
right_gaze_pt_y = gaze_ctrs['right_eye']['y_center']
print("left_gaze_pt_x:", left_gaze_pt_x)
print("left_gaze_pt_y:", left_gaze_pt_y)
print("right_gaze_pt_x:", right_gaze_pt_x)
print("right_gaze_pt_y:", right_gaze_pt_y)

left_gaze_pt_x: 411
left_gaze_pt_y: 235
right_gaze_pt_x: 151
right_gaze_pt_y: 246


In [15]:
# Normalized Point (z=1.0)
left_undistort_pts = cv2.undistortPoints((left_gaze_pt_x, left_gaze_pt_y), 
                                         np.array(cam_mtxs['left_eye']['camera_matrix']), 
                                         np.array(cam_mtxs['left_eye']['distortion_coefficients']))
right_undistort_pts = cv2.undistortPoints((right_gaze_pt_x, right_gaze_pt_y), 
                                          np.array(cam_mtxs['right_eye']['camera_matrix']), 
                                          np.array(cam_mtxs['right_eye']['distortion_coefficients']))
print('left_undistort_pts:', left_undistort_pts)  # For Y, Downwards is + because of Camera Opencv Coordinate System
print('right_undistort_pts:', right_undistort_pts)  # For Y, Downwards is + because of Camera Opencv Coordinate System

left_undistort_pts: [[[0.19142607 0.03459041]]]
right_undistort_pts: [[[-0.2600561   0.05559228]]]


In [16]:
# Undistored Points

left_x_prime = left_undistort_pts.squeeze()[0]
left_y_prime = left_undistort_pts.squeeze()[1]
right_x_prime = right_undistort_pts.squeeze()[0]
right_y_prime = right_undistort_pts.squeeze()[1]

print('left_x_prime:',left_x_prime)
print('left_y_prime:',left_y_prime)
print('right_x_prime:',right_x_prime)
print('right_y_prime:',right_y_prime)

left_x_prime: 0.19142607185001126
left_y_prime: 0.034590406011504184
right_x_prime: -0.26005610414060365
right_y_prime: 0.055592279312658976


In [17]:
# Checking of Undistortion
left_pixels,_ = cv2.projectPoints((left_undistort_pts.squeeze()[0], left_undistort_pts.squeeze()[1], 1.0), 
                                np.array([0.0,0.0,0.0]), np.array([0.0,0.0,0.0]), 
                                  np.array(cam_mtxs['left_eye']['camera_matrix']), 
                                  np.array(cam_mtxs['left_eye']['distortion_coefficients']))
right_pixels,_ = cv2.projectPoints((right_undistort_pts.squeeze()[0], right_undistort_pts.squeeze()[1], 1.0), 
                                 np.array([0.0,0.0,0.0]), np.array([0.0,0.0,0.0]), 
                                  np.array(cam_mtxs['right_eye']['camera_matrix']), 
                                  np.array(cam_mtxs['right_eye']['distortion_coefficients']))
print('left_pixels:', left_pixels)
print('right_pixels:',right_pixels)

left_pixels: [[[411. 235.]]]
right_pixels: [[[151.00000002 246.        ]]]


## Orientation Offset Calculation

In [18]:
left_yaw_offset = math.atan2(left_undistort_pts.squeeze()[0],1.0)  # radians unit
left_pitch_offset = math.atan2(left_undistort_pts.squeeze()[1],1.0) # For Y, Downwards is + because of Camera Opencv Coordinate System
right_yaw_offset = math.atan2(right_undistort_pts.squeeze()[0],1.0) 
right_pitch_offset = math.atan2(right_undistort_pts.squeeze()[1],1.0) # For Y, Downwards is + because of Camera Opencv Coordinate System

print('left_yaw_offset(rad):', left_yaw_offset)
print('left_pitch_offset(rad)(opencv coord):', left_pitch_offset)
print('right_yaw_offset:(rad)', right_yaw_offset)
print('right_pitch_offset(rad)(opencv coord):', right_pitch_offset)
print('-----')

print('left_yaw_offset(deg):', math.degrees(left_yaw_offset))
print('left_pitch_offset(deg)(opencv coord):', math.degrees(left_pitch_offset))
print('right_yaw_offset(deg):', math.degrees(right_yaw_offset))
print('right_pitch_offset(deg)(opencv coord):', math.degrees(right_pitch_offset))

left_yaw_offset(rad): 0.18913797017088096
left_pitch_offset(rad)(opencv coord): 0.03457662014401592
right_yaw_offset:(rad) -0.2544206094843146
right_pitch_offset(rad)(opencv coord): 0.05553511593216458
-----
left_yaw_offset(deg): 10.836807436462736
left_pitch_offset(deg)(opencv coord): 1.981094404079137
right_yaw_offset(deg): -14.57722714459731
right_pitch_offset(deg)(opencv coord): 3.1819277576827667


# Generate Dataset

## Proving

In [19]:
mat = tm.get_transform('realsense','leftcamera')
mat

array([[ 0.96074191, -0.0361809 ,  0.27507441, -0.03027039],
       [ 0.0388817 ,  0.99923427, -0.00437003,  0.01677771],
       [-0.27470566,  0.01489383,  0.96141301, -0.25933008],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [20]:
# Example

in_pts = np.array([0.5,0.8,0.75,1]).reshape(-1,1)
out_pts = mat @ in_pts
print('in_pts:',in_pts)
print('out_pts:',out_pts)

in_pts: [[0.5 ]
 [0.8 ]
 [0.75]
 [1.  ]]
out_pts: [[0.62746165]
 [0.83232845]
 [0.33629191]
 [1.        ]]


In [21]:
# Solution
x_prime = out_pts[0,0]/out_pts[2,0]
y_prime = out_pts[1,0]/out_pts[2,0]
a = np.array([[mat[0,0],mat[0,1],-x_prime],
              [mat[1,0],mat[1,1],-y_prime],
              [mat[2,0],mat[2,1],-1]])
b = np.array([-mat[0,2]*in_pts[2,0]-mat[0,3], -mat[1,2]*in_pts[2,0]-mat[1,3], -mat[2,2]*in_pts[2,0]-mat[2,3]])
x = np.linalg.solve(a, b)
x

array([0.5       , 0.8       , 0.33629191])

In [22]:
# Function Prototype

x_prime = out_pts[0,0]/out_pts[2,0]
y_prime = out_pts[1,0]/out_pts[2,0]
z = in_pts[2,0]

a = np.array([[mat[0,0],mat[0,1],-x_prime],
              [mat[1,0],mat[1,1],-y_prime],
              [mat[2,0],mat[2,1],-1]])
b = np.array([-mat[0,2]*z-mat[0,3], -mat[1,2]*z-mat[1,3], -mat[2,2]*z-mat[2,3]])
x = np.linalg.solve(a, b)
x

array([0.5       , 0.8       , 0.33629191])

In [23]:
def generate_chest_dataset(mat, z, x_prime, y_prime):
    a = np.array([[mat[0,0],mat[0,1],-x_prime],
              [mat[1,0],mat[1,1],-y_prime],
              [mat[2,0],mat[2,1],-1]])
    b = np.array([-mat[0,2]*z-mat[0,3], -mat[1,2]*z-mat[1,3], -mat[2,2]*z-mat[2,3]])
    x = np.linalg.solve(a, b)
    return np.append(x[:2],z)

In [24]:
# Testing the Function
generate_chest_dataset(mat=mat, z=in_pts[2,0], x_prime=out_pts[0,0]/out_pts[2,0], y_prime=out_pts[1,0]/out_pts[2,0])

array([0.5 , 0.8 , 0.75])

## Data Generation (Single)

In [25]:
# Initialization
z_depth = 1.0

# Motor Command
joint_dict = motor_cmd_joint(cmd_lnp=10,cmd_lnt=10,cmd_unt=10,cmd_et=10,cmd_lep=5,cmd_rep=5)
tm.set_joint('neck_pitch', joint_dict['neck_pitch'])
tm.set_joint('neck_yaw', joint_dict['neck_yaw'])
tm.set_joint('head_pitch', joint_dict['head_pitch'])
tm.set_joint('eyes_pitch', joint_dict['eyes_pitch'])
tm.set_joint('lefteye_yaw', joint_dict['lefteye_yaw'])
tm.set_joint('righteye_yaw', joint_dict['righteye_yaw'])

# Get Transform
T_clprime = tm.get_transform('realsense','leftcamera')
T_crprime = tm.get_transform('realsense','rightcamera')

# Get Camera Chest Points
X_C_L = generate_chest_dataset(mat=T_clprime, z=z_depth, x_prime=left_x_prime, y_prime=left_y_prime)
Y_C_L = generate_chest_dataset(mat=T_crprime, z=z_depth, x_prime=right_x_prime, y_prime=right_y_prime)
print('X_C_L:',X_C_L)
print('Y_C_L:',Y_C_L)

X_C_L: [-0.10829473  0.01713932  1.        ]
Y_C_L: [-0.44905235  0.10359873  1.        ]


## Data Generation (Multiple)

In [30]:
# Robot Construction
robot = URDF.from_xml_file(urdf_path)

# URDF Variable Assignment
for joint in robot.joints:
    if joint.name == 'torso':
        joint.origin.position[0] = -0.12234 + 0  # Offset
        joint.origin.position[1] = -0.0325 + 0
        joint.origin.position[2] = 0.05692 + 0
    elif joint.name == 'neck_pitch':
        joint.origin.rotation[0] = 0 + 0
    elif joint.name == 'head_pitch':
        joint.origin.position[2] = 0.13172 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[2] = 0 + 0
    elif joint.name == 'eyes_pitch':
        joint.origin.position[0] = 0.08492 + 0
        joint.origin.position[2] = 0.05186 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[2] = 0 + 0
    elif joint.name == 'lefteye_yaw':
        joint.origin.position[1] = 0.02895 + 0
    elif joint.name == 'righteye_yaw':
        joint.origin.position[1] = -0.02895 + 0
    elif joint.name == 'lefteye_cam':
        joint.origin.position[0] = 0.015 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[1] = 0 + 0.03457662014401592*(-1)  # For Y, Downwards is + because of Camera Opencv Coordinate System
        joint.origin.rotation[2] = 0 + 0.18913797017088096  # Orientation Offset: Right Hand Rule (Positive CCW)
    elif joint.name == 'righteye_cam':
        joint.origin.position[0] = 0.015 + 0
        joint.origin.rotation[0] = 0 + 0
        joint.origin.rotation[1] = 0 + 0.05553511593216458*(-1)  # For Y, Downwards is + because of Camera Opencv Coordinate System
        joint.origin.rotation[2] = 0 + -0.2544206094843146  # Orientation Offset: Right Hand Rule (Positive CCW)
        
# XML to String
urdf_str = xml_to_str(robot)

# Transform Manager
tm = UrdfTransformManager()
tm.load_urdf(urdf_str)

In [26]:
# Initialization
z_depth = 1.0
list_ep=list(range(-14,15,2))
list_et=list(range(20,-31,-5))
list_lnp=list(range(-35,36,5))
list_lnt=list(range(-10,31,10))
list_unt=list(range(40,-11,-10))
data_dict = { 'x_c_l': [],
              'y_c_l': [],
              'z_c_l': [],
              'x_c_r': [],
              'y_c_r': [],
              'z_c_r': [],
              'cmd_theta_lower_neck_pan':[],
              'cmd_theta_lower_neck_tilt':[],
              'cmd_theta_upper_neck_tilt':[],
              'cmd_theta_left_eye': [],
              'cmd_theta_right_eye': [],
              'cmd_theta_tilt':[],
              'neck_pitch': [],
              'neck_yaw': [],
              'head_pitch': [],
              'eyes_pitch': [],
              'lefteye_yaw': [],
              'righteye_yaw': [],
            }

# Data Generation
for lnt in list_lnt:
    for unt in list_unt:
        for lnp in list_lnp:
            for et in list_et:
                for ep in list_ep:
                    # Robot Commands
                    joint_dict = motor_cmd_joint(cmd_lnp=lnp,cmd_lnt=lnt,cmd_unt=unt,cmd_et=et,cmd_lep=ep,cmd_rep=ep)
                    neck_pitch = joint_dict['neck_pitch']
                    neck_yaw = joint_dict['neck_yaw']
                    head_pitch = joint_dict['head_pitch']
                    eyes_pitch = joint_dict['eyes_pitch']
                    lefteye_yaw = joint_dict['lefteye_yaw']
                    righteye_yaw = joint_dict['righteye_yaw']

                    # Setting Robot Joints
                    tm.set_joint('neck_pitch', neck_pitch)
                    tm.set_joint('neck_yaw', neck_yaw)
                    tm.set_joint('head_pitch', head_pitch)
                    tm.set_joint('eyes_pitch', eyes_pitch)
                    tm.set_joint('lefteye_yaw', lefteye_yaw)
                    tm.set_joint('righteye_yaw', righteye_yaw)
                    
                    # Get Transform
                    T_clprime = tm.get_transform('realsense','leftcamera')
                    T_crprime = tm.get_transform('realsense','rightcamera')
                
                    # Get Camera Chest Points
                    X_C_L = generate_chest_dataset(mat=T_clprime, z=z_depth, x_prime=left_x_prime, y_prime=left_y_prime)
                    Y_C_L = generate_chest_dataset(mat=T_crprime, z=z_depth, x_prime=right_x_prime, y_prime=right_y_prime)

                    # Saving Data
                    data_dict['x_c_l'].append(X_C_L[0])
                    data_dict['y_c_l'].append(X_C_L[1])
                    data_dict['z_c_l'].append(X_C_L[2])
                    data_dict['x_c_r'].append(Y_C_L[0])
                    data_dict['y_c_r'].append(Y_C_L[1])
                    data_dict['z_c_r'].append(Y_C_L[2])
                    data_dict['cmd_theta_lower_neck_pan'].append(lnp)
                    data_dict['cmd_theta_lower_neck_tilt'].append(lnt)
                    data_dict['cmd_theta_upper_neck_tilt'].append(unt)
                    data_dict['cmd_theta_left_eye'].append(ep)
                    data_dict['cmd_theta_right_eye'].append(ep)
                    data_dict['cmd_theta_tilt'].append(et)
                    data_dict['neck_pitch'].append(neck_pitch)
                    data_dict['neck_yaw'].append(neck_yaw)
                    data_dict['head_pitch'].append(head_pitch)
                    data_dict['eyes_pitch'].append(eyes_pitch)
                    data_dict['lefteye_yaw'].append(lefteye_yaw)
                    data_dict['righteye_yaw'].append(righteye_yaw)

In [27]:
# Pandas Dataframe

data_df = pd.DataFrame(data_dict)
print(len(data_df))
data_df.head()

74250


x_c_l     y_c_l  z_c_l     x_c_r     y_c_r  z_c_r   
0 -0.275052  0.020852    1.0 -0.636376 -0.239563    1.0  \
1 -0.268397  0.016812    1.0 -0.639778 -0.222736    1.0   
2 -0.262015  0.012446    1.0 -0.641922 -0.205682    1.0   
3 -0.255921  0.007765    1.0 -0.642802 -0.188491    1.0   
4 -0.250128  0.002785    1.0 -0.642423 -0.171252    1.0   

   cmd_theta_lower_neck_pan  cmd_theta_lower_neck_tilt   
0                       -35                        -10  \
1                       -35                        -10   
2                       -35                        -10   
3                       -35                        -10   
4                       -35                        -10   

   cmd_theta_upper_neck_tilt  cmd_theta_left_eye  cmd_theta_right_eye   
0                         40                 -14                  -14  \
1                         40                 -12                  -12   
2                         40                 -10                  -10   
3                         40                  -8                   -8   
4                         40                  -6                   -6   

   cmd_theta_tilt  neck_pitch  neck_yaw  head_pitch  eyes_pitch  lefteye_yaw   
0              20   -0.087266 -0.305433    0.349066    0.139626    -0.390954  \
1              20   -0.087266 -0.305433    0.349066    0.139626    -0.335103   
2              20   -0.087266 -0.305433    0.349066    0.139626    -0.279253   
3              20   -0.087266 -0.305433    0.349066    0.139626    -0.223402   
4              20   -0.087266 -0.305433    0.349066    0.139626    -0.167552   

   righteye_yaw  
0     -0.415388  
1     -0.356047  
2     -0.296706  
3     -0.237365  
4     -0.178024

In [29]:
# Saving CSV

sim_data_path = os.path.join(os.getcwd(), '..','data','sim','240917_sim_dataset.csv')
data_df.to_csv(sim_data_path , index=False)
print('Saved to:',sim_data_path)

Saved to: /home/jaynieles/dev/grace_dl/notebooks/../data/sim/240917_sim_dataset.csv
